In [1]:
# Imports you may need
from dataloader import *
from finance import *
import pandas as pd
import pageviewapi 

from finance import stock, compare
from quotebankexploration import *
from wikipedia import *

%load_ext autoreload
%autoreload 2

In [3]:
# Get some data frames
wiki_data = concat_wiki_files()
filtered_quotes = get_filtered_quotes()
speakers_id = pd.DataFrame(get_speakers_ids(filtered_quotes))

In [26]:
'''
REMARK: ici, tu changes le numéro de ton cluster. Je m'explique, tu as 4
clusters, donc tu mets un chiffre parmi {1, 2, 3, 4} selon le cluster que
tu lances. (ex: le premier tu mets cluster = 1, le deuxième tu mets
cluster = 2, etc...)
'''
cluster = 1

# DO NOT TOUCH
speakers_label = add_labels(speakers_id, wiki_data, save = True, cluster = cluster)